# Getting Started with Keypoint Estimation: Loading Keypoint Data



### Loading Keypoint Data in COCO Format

Start by downloading the dataset from [this website](https://zenodo.org/records/10057090). Alternatively, you can use your favorite method for programmatically downloading the dataset, for example by running the following command in your terminal (Note: This is a 6GB download):

```bash
wget https://zenodo.org/records/10057090/files/tampar.zip?download=1
```

This dataset is in [COCO format](https://beta-docs.voxel51.com/fiftyone_concepts/dataset_creation/datasets/#cocodetectiondataset) and we can [automatically load it FiftyOne format](https://beta-docs.voxel51.com/fiftyone_concepts/dataset_creation/#common-formats).

In [ ]:
!pip install pycocotools

This code demonstrates how to load and prepare the TAMPAR dataset using FiftyOne's data management capabilities. Here's what's happening:


- **[`fo.Dataset.from_dir()`](https://beta-docs.voxel51.com/api/fiftyone.core.dataset.Dataset.html#from_dir)**: This method imports the TAMPAR dataset in COCO format directly from disk. It accepts several important parameters:
  - `dataset_type=fo.types.COCODetectionDataset`: Specifies we're loading data in [COCO object detection format](https://beta-docs.voxel51.com/api/fiftyone.types.dataset_types.COCODetectionDataset.html)
  - `data_path` and `labels_path`: Point to the directories containing images and annotation JSON file
  - `include_id=True`: Preserves the original COCO IDs
  - `name="TAMPAR"`: Gives our dataset a [descriptive name](https://beta-docs.voxel51.com/api/fiftyone.core.dataset.Dataset.html#name) in FiftyOne
  - `persistent=True`: [Persists](https://beta-docs.voxel51.com/fiftyone_concepts/using_datasets/#dataset-persistence) the dataset to disk for future sessions

- **[`compute_metadata()`](https://beta-docs.voxel51.com/api/fiftyone.core.collections.SampleCollection.html#compute_metadata)**: After loading, this method analyzes all images to extract and store metadata like dimensions, file sizes, and other properties that enhance the dataset's functionality within FiftyOne.


In [59]:
import fiftyone as fo

tampar_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path="tampar",
    labels_path="tampar/tampar_test.json",
    include_id=True,
    name="TAMPAR",
    persistent=True
)

tampar_dataset.compute_metadata()

 100% |█████████████████| 485/485 [26.2s elapsed, 0s remaining, 20.0 samples/s]      


The keypoints are automatically parsed as [Keypoint labels](https://beta-docs.voxel51.com/api/fiftyone.core.labels.Keypoint.html) in the Dataset. 

Keypoints in FiftyOne represents a collection of coordinate points that mark specific locations in an image. These can be used for localizing important features like facial landmarks, human pose joints, or in our case, the corners of parcel boxes. Each [Keypoint label](https://beta-docs.voxel51.com/api/fiftyone.core.labels.Keypoint.html) contains:

* A set of points as (x,y) coordinates normalized to [0,1] range
* Optional confidence scores for each point
* A semantic label describing what these points represent
* Optional attributes for storing additional metadata

This dataset has 8 keypoints (24 values total), all with `visibility=2` (which means "visible")

In [62]:
tampar_dataset.first()['keypoints']

<Keypoints: {
    'keypoints': [
        <Keypoint: {
            'id': '67d30c7fba3cc01259e1f17c',
            'attributes': {},
            'tags': [],
            'label': 'normal box',
            'points': [
                [0.5589583333333333, 0.6946957671957673],
                [0.14292162698412697, 0.4266931216931217],
                [0.09013640873015873, 0.21361772486772487],
                [0.5546006944444445, 0.9186904761904762],
                [0.898640873015873, 0.31390873015873016],
                [0.4302876984126984, 0.27409391534391536],
                [0.43548611111111113, 0.06962632275132276],
                [0.8419171626984128, 0.5316666666666666],
            ],
            'confidence': None,
            'index': None,
            'visible': [2, 2, 2, 2, 2, 2, 2, 2],
            'supercategory': 'box',
            'iscrowd': 0,
            'num_keypoints': 8,
            'occluded': False,
        }>,
    ],
}>

Now that the Dataset is loaded, we can [launch the app](https://beta-docs.voxel51.com/getting_started/basic/application_tour/) and inspect it.

```python
fo.launch_app(tampar_dataset)
```

<img src="assets/tampar.gif">

# Connecting Keypoint Edges in FiftyOne

When combined with a [`KeypointSkeleton`](https://beta-docs.voxel51.com/api/fiftyone.core.odm.dataset.KeypointSkeleton.html), these individual points form a connected structure (like our box wireframe) that visually represents the spatial relationships between points.

In our parcel detection dataset, we use keypoints to mark the 8 corners of each box, allowing us to reconstruct the 3D geometry of parcels from 2D images.

## The Skeleton Structure

All [Dataset](https://beta-docs.voxel51.com/api/fiftyone.core.dataset.html#fiftyone.core.dataset.Dataset) instances have [`skeletons`](https://beta-docs.voxel51.com/api/fiftyone.core.dataset.Dataset.html#skeletons) and [`default_skeleton`](https://beta-docs.voxel51.com/api/fiftyone.core.dataset.Dataset.html#default_skeleton) properties that you can use to store keypoint skeletons for Keypoint field(s) of a dataset.

To visualize these boxes correctly in FiftyOne, we define a skeleton that connects these keypoints with edges. Our [`KeypointSkeleton`](https://beta-docs.voxel51.com/api/fiftyone.core.odm.dataset.KeypointSkeleton.html) defines:

1. **Labels**: Semantic names for each keypoint (e.g., "front_top_right", "back_bottom_left")
2. **Edges**: Connections between pairs of keypoints that form the wireframe of the 3D box

## Edge Connections

The edges are organized to represent the physical structure of the keypoints:

- **Front Face**: Connects the four front corners in a clockwise or counter-clockwise order

- **Top Face**: Connects the four top corners (front-top-left → back-top-left → back-top-right → front-top-right)

- **Right Side**: Connects the front and back edges on the right side of the box

- **Additional Edges**: Completes the remaining connections to form a full box

## Implementation

In FiftyOne, we implement this by creating a [`KeypointSkeleton`](https://beta-docs.voxel51.com/api/fiftyone.core.odm.dataset.KeypointSkeleton.html) object:


In [60]:
import fiftyone as fo

labels = [
    "front_upper_right",    # 0
    "front_lower_left",     # 1
    "front_upper_left",     # 2
    "front_lower_right",    # 3
    "back_upper_right",     # 4
    "back_upper_left",      # 5 
    "back_upper_left",      # 6 
    "back_lower_right"      # 7
]

# Complete set of edges to create a full 3D box
edges = [
    # Front face - complete square
    [2, 0],  # front upper-left to front upper-right
    [0, 3],  # front upper-right to front lower-right
    [3, 1],  # front lower-right to front lower-left
    [1, 2],  # front lower-left to front upper-left
    
    # Top face
    [2, 6],  # front upper-left to back upper-left
    [6, 4],  # back upper-left to back upper-right
    [4, 0],  # back upper-right to front upper-right
    
    # Right side face
    [0, 4],  # front upper-right to back upper-right
    [4, 7],  # back upper-right to back lower-right
    [7, 3],  # back lower-right to front lower-right
    
    # Back face 
    [6, 4],  # back upper-left to back upper-right
    [4, 7],  # back upper-right to back lower-right
    
    # Left side face 
    [2, 6],  # front upper-left to back upper-left
    
    # Needed for complete box
    [1, 5],  # front lower-left to back lower-left
    [5, 6],  # back lower-left to back upper-left
    [5, 7]   # back lower-left to back lower-right
]


tampar_dataset.default_skeleton = fo.KeypointSkeleton(
    labels=labels,
    edges=edges
)

tampar_dataset.save()



This skeleton definition ensures that when our 3D box keypoints are visualized in FiftyOne, the lines connecting them accurately represent the box's structure, making it easy to interpret the detection results at a glance.


```python
fo.launch_app(tampar_dataset)
```

<img src="assets/tampar-skeletons.gif">



### Next steps

* Checkout these poset estimation datasets on the Hugging Face Hub:
  * [DensePose-COCO](https://huggingface.co/datasets/Voxel51/DensePose-COCO)
  * [MPII_Human_Pose_Dataset](https://huggingface.co/datasets/Voxel51/MPII_Human_Pose_Dataset)

* Learn about working with [Detections](https://beta-docs.voxel51.com/how_do_i/recipes/adding_detections/)

* Learn about working with [Segmentations](https://beta-docs.voxel51.com/fiftyone_concepts/using_datasets/#semantic-segmentation)

* Join the [Discord community](https://community.voxel51.com/)

* Follow us on [LinkedIn](https://www.linkedin.com/company/voxel51/)